# A session of C++ showing functional programming with REPL

### Links to other demos
* [A session of REPL / C++ with opencv](opencv_example.ipynb) (notebook here)
* [A session of C++ showing functional programming with REPL](Functional_REPL.ipynb) (notebook here)
* [Advices and Gotchas when using cling and jupyter](Advices_And_Gotchas.ipynb) (notebook here)
* [Interactive C++ REPL session from the console](https://github.com/pthom/Cling_Repl_Demo/blob/master/examples/shell_cling/Shell_Functional_REPL.md) (web page on github)
* [Main page](https://github.com/pthom/Cling_Repl_Demo/blob/master/Readme.md) (web page on github)

#### How to install these demos on a docker container

See https://github.com/pthom/Cling_Repl_Demo/tree/master/Docker_xeus


## About FunctionalPlus

[FunctionalPlus](https://github.com/Dobiasd/FunctionalPlus) (also know as `fplus`) is a Functional Programming Library for C++. 
It feature an [api search engine](http://www.editgym.com/fplus-api-search/), which is inspired by [hoogle, haskell's api search engine](https://www.haskell.org/hoogle/).


---------------

# Demo

**Important note**

if you are reading this inside jupyter notebook:
* Click on the "Run" button at the top in order to run each cell (or type "Shift + Enter" in order to run it, and go to the next cell)
* For a better undertsanding of this page, it is better to clear the outputs below: click on the "Kernel" menu, and select "Restart & Clear Output"



# Include fplus header

(fplus is a header only library)

In [1]:
#pragma cling add_include_path("../../external/FunctionalPlus/include")
#include <fplus/fplus.hpp>

# Read some text
Let's try to read some text.

Note: If you do not end your line with a ";", cling will output the result of the last computation

In [ ]:
using namespace std;
fplus::read_text_file("../../data/if.txt") // no ";"

Wow, where is our text ? fplus::read_text_file did not return a string !

The reason for this is that fplus is a functional library,
so that fplus::read_text_file(filename) does not perform the side effect :
instead it returns a function which you need to invoke
in order to perform the side effect.
    
Let's try again, and invoke the function : see the () at the end below

In [ ]:
auto poem = fplus::read_text_file("../../data/if.txt")();
poem

# Split the text into lines

In [ ]:
// Let's try to split some lines
const auto lines = fplus::split_lines(false, poem);

In [ ]:
lines

# Modify the text word by word
Let's try to modify this poem, by applying a function
that changes all letter of each word to lowercase, except the first letter of each word.

In [ ]:
// This is the function we want to apply to each word
std::string capitalize_first_letter(const std::string & word) {
    auto result = fplus::to_lower_case(word);
    result[0] = toupper(result[0]);
    return result;
}

capitalize_first_letter("hello")

## Let's start with the first line
### First attempt : step by step
With the code below, we can split the first line into words.

In [ ]:
const std::string first_line = lines[0];
const auto words = fplus::split<std::string>(' ', false, first_line);
words

How to transform all these words using `capitalize_first_letter`? 
We will be using `fplus::transform` : it applies a given transformation to all the elements of a container.
See it's documentation at http://www.editgym.com/fplus-api-search/

In [ ]:
auto words_transformed = fplus::transform(capitalize_first_letter, words);
words_transformed

Then, we need to join our transformed words:

In [ ]:
auto first_line_transformed = fplus::join(std::string(" "), words_transformed);
first_line_transformed

So, the final result for the first line could be written:

In [ ]:
fplus::join( std::string(" "), 
             fplus::transform(capitalize_first_letter, 
                              fplus::split<std::string>(' ', false, first_line) 
                             )  
           )

### Second attempt 
`apply_by_words` is a higher order function that will transform a function f into another function
 that will appply f word by word

In [ ]:
auto apply_by_words = [](auto f) {
    return fplus::fwd::compose(
        fplus::fwd::split(' ', false),
        fplus::fwd::transform(f),
        fplus::fwd::join(std::string(" "))
    );
 };;  

Note : the double ";;" after the lambda definition is important
This is a known bug in cling : see [Gotchas.ipynb](Gotchas.ipynb)

In [ ]:
// Now let's instantiate apply_by_words with capitalize_first_letter
// cap_words will be a lambda function of type : string -> string
auto cap_words = apply_by_words(capitalize_first_letter);

In [ ]:
// And let's try it
cap_words(lines[0])

## And now, let's transform all the lines
`apply_by_lines` is another higher order function that will transform a function f into another function
 that will appply f word by word

In [ ]:
// Let's continue
auto apply_by_lines = [](auto f) {
    return fplus::fwd::compose(
        fplus::fwd::split_lines(false),
        fplus::fwd::transform(f),
        fplus::fwd::join(std::string("\n"))
    );
 };; // the double ;; is voluntary here (bug in cling !)


In [ ]:
auto cap_text = apply_by_lines(cap_words);;

In [ ]:
// And now let's apply this to the complete poem
cap_text(poem)

# Let's make it a program : use fplus::interact

`fplus::interact`is a higher order function that transforms a function string -> string into a program that read it's output from stdin and writes it output to stdout.



In [ ]:
auto prog = fplus::interact(cap_text);

Let's examine prog:

In [ ]:
prog

So, `prog` is itself a lambda. In order to construct a program, we need to call it inside main, like so: 

In [ ]:
int main() {
    prog();
}

So, a full program that would apply our transformation to stdin and write to stdout could be written as below.
(beware, this code can not be used inside this page, you need to copy / paste it into a cpp file).

````cpp
#include <fplus/fplus.hpp>

std::string capitalize_first_letter(const std::string & word) {
    auto result = fplus::to_lower_case(word);
    result[0] = toupper(result[0]);
    return result;
}

int main()
{
  auto apply_by_words = [](auto f) {
    return fplus::fwd::compose(
        fplus::fwd::split(' ', false),
        fplus::fwd::transform(f),
        fplus::fwd::join(std::string(" "))
    );
   };
  auto apply_by_lines = [](auto f) {
      return fplus::fwd::compose(
          fplus::fwd::split_lines(false),
          fplus::fwd::transform(f),
          fplus::fwd::join(std::string("\n"))
      );
  };

  auto cap_words = apply_by_words(capitalize_first_letter);
  auto cap_text = apply_by_lines(cap_words);
  auto prog = fplus::interact(cap_text);

  prog();
}
````